In [17]:
%pip install pyvis



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from pathlib import Path
import json


In [19]:
# ============ Configuration ============
INPUT_CSV = "hp-dialogues.csv"

# ============ Data Loading ============
print("📊 Loading dialogue data...")
df = pd.read_csv(INPUT_CSV)

# Lowercase speaker and target names for consistency
print("🔄 Converting speaker and target names to lowercase...")
df['speaker'] = df['speaker'].str.lower()
df['target'] = df['target'].str.lower()

print(f"✅ Loaded {len(df)} dialogue entries")
print(f"📝 Unique speakers: {df['speaker'].nunique()}")
print(f"🎯 Unique targets: {df['target'].nunique()}")
df.head()


📊 Loading dialogue data...
🔄 Converting speaker and target names to lowercase...
✅ Loaded 28643 dialogue entries
📝 Unique speakers: 212
🎯 Unique targets: 428


book    speaker         target sentiment
0     7     yaxley  severus snape   neutral
1     7     yaxley  lucius malfoy  negative
2     7  voldemort         yaxley   neutral
3     7  voldemort  severus snape   neutral
4     7  voldemort  severus snape  positive

In [20]:
# ============ Data Aggregation ============
print("🔄 Aggregating dialogue data by speaker-target pairs...")

# Collapse edges (speaker → target): total weight + dominant sentiment
agg = df.groupby(["speaker", "target"]).agg(
    weight=("sentiment", "size"),
    pos_count=("sentiment", lambda x: (x == "positive").sum()),
    neg_count=("sentiment", lambda x: (x == "negative").sum()),
    neu_count=("sentiment", lambda x: (x == "neutral").sum()),
).reset_index()

def pick_sentiment(row):
    """Determine dominant sentiment for a relationship"""
    counts = {"positive": row["pos_count"], "negative": row["neg_count"], "neutral": row["neu_count"]}
    mx = max(counts.values())
    winners = [s for s, c in counts.items() if c == mx]
    # Priority: negative > positive > neutral (in case of ties)
    if "negative" in winners: return "negative"
    if "positive" in winners: return "positive"
    return "neutral"

agg["sentiment"] = agg.apply(pick_sentiment, axis=1)

print(f"✅ Created {len(agg)} unique speaker-target relationships")
agg.head(10)


🔄 Aggregating dialogue data by speaker-target pairs...
✅ Created 3630 unique speaker-target relationships


speaker                 target  weight  pos_count  neg_count  \
0  aberforth dumbledore   aberforth dumbledore       1          1          0   
1  aberforth dumbledore       albus dumbledore      12          0         11   
2  aberforth dumbledore      ariana dumbledore       6          0          5   
3  aberforth dumbledore  death eater (dolohov)       1          0          0   
4  aberforth dumbledore           death eaters       9          0          8   
5  aberforth dumbledore              dementors       1          0          1   
6  aberforth dumbledore                  dobby       1          1          0   
7  aberforth dumbledore           elphias doge       2          0          1   
8  aberforth dumbledore    gellert grindelwald       5          0          4   
9  aberforth dumbledore          ginny weasley       1          1          0   

   neu_count sentiment  
0          0  positive  
1          1  negative  
2          1  negative  
3          1   neutral  
4          1  negative  
5          0  negative  
6          0  positive  
7          1  negative  
8          1  negative  
9          0  positive

In [21]:
# ============ Graph Setup ============
print("🕸️ Building network graph and calculating layout...")

# Build graph (for layout)
G = nx.from_pandas_edgelist(
    agg, source="speaker", target="target", edge_attr=["sentiment", "weight"], create_using=nx.DiGraph()
)

# Calculate positions using spring layout
pos = nx.spring_layout(G, weight="weight", k=5.0, iterations=150, seed=42)

print(f"✅ Graph created with {len(G.nodes())} nodes and {len(G.edges())} edges")


🕸️ Building network graph and calculating layout...
✅ Graph created with 440 nodes and 3630 edges


In [22]:
# ============ Graph Data Builder Function ============
def remarks_to_color(neg, pos, neu):
    """Convert sentiment counts to RGB color using weighted color mixing."""
    total = pos + neg + neu
    if total == 0:
        return "rgb(128,128,128)"  # Default gray
    
    # Normalize to percentages
    pos_pct = pos / total
    neg_pct = neg / total
    neu_pct = neu / total
    
    # Define base colors
    colors = {
        'positive': {'r': 34, 'g': 197, 'b': 94},   # Green
        'negative': {'r': 239, 'g': 68, 'b': 68},   # Red
        'neutral': {'r': 107, 'g': 114, 'b': 128}   # Gray
    }
    
    # Calculate weighted color mixing
    r = round(
        colors['positive']['r'] * pos_pct + 
        colors['negative']['r'] * neg_pct + 
        colors['neutral']['r'] * neu_pct
    )
    g = round(
        colors['positive']['g'] * pos_pct + 
        colors['negative']['g'] * neg_pct + 
        colors['neutral']['g'] * neu_pct
    )
    b = round(
        colors['positive']['b'] * pos_pct + 
        colors['negative']['b'] * neg_pct + 
        colors['neutral']['b'] * neu_pct
    )
    
    return f"rgb({r},{g},{b})"

def build_graph_data(mode: str, max_book: int = 7):
    """Return nodes + edges for a given mode (speaker/target) up to a specific book."""
    # Filter dataframe to only include data up to max_book
    df_filtered = df[df["book"] <= max_book]
    
    # Create filtered aggregation data
    agg_filtered = df_filtered.groupby(["speaker", "target"]).agg(
        weight=("sentiment", "size"),
        pos_count=("sentiment", lambda x: (x == "positive").sum()),
        neg_count=("sentiment", lambda x: (x == "negative").sum()),
        neu_count=("sentiment", lambda x: (x == "neutral").sum()),
    ).reset_index()
    
    def pick_sentiment(row):
        """Determine dominant sentiment for a relationship"""
        counts = {"positive": row["pos_count"], "negative": row["neg_count"], "neutral": row["neu_count"]}
        mx = max(counts.values())
        winners = [s for s, c in counts.items() if c == mx]
        # Priority: negative > positive > neutral (in case of ties)
        if "negative" in winners: return "negative"
        if "positive" in winners: return "positive"
        return "neutral"

    agg_filtered["sentiment"] = agg_filtered.apply(pick_sentiment, axis=1)
    
    if mode == "speaker":
        counts = df_filtered["speaker"].value_counts().to_dict()
        def comments_of(char): return df_filtered[df_filtered["speaker"] == char]
        title_prefix = "remarks made"
    else:
        counts = df_filtered["target"].value_counts().to_dict()
        def comments_of(char): return df_filtered[df_filtered["target"] == char]
        title_prefix = "remarks received"

    all_chars = set(df_filtered["speaker"].unique()) | set(df_filtered["target"].unique())
    node_colors = {}
    for char in all_chars:
        cc = comments_of(char)
        pos_c = (cc["sentiment"] == "positive").sum()
        neg_c = (cc["sentiment"] == "negative").sum()
        neu_c = (cc["sentiment"] == "neutral").sum()
        
        node_colors[char] = remarks_to_color(neg_c, pos_c, neu_c)

    max_count = max(counts.values()) if counts else 1
    min_size, max_size = 10, 50

    nodes = []
    for node in G.nodes():
        ccount = counts.get(node, 0)
        size = min_size + (max_size - min_size) * (ccount / max_count)
        color = node_colors.get(node, "rgb(128,128,128)")
        # Use position if available, otherwise default to (0,0)
        if node in pos:
            x, y = pos[node]
        else:
            x, y = (0, 0)
        
        # Calculate sentiment percentages for tooltip
        cc = comments_of(node)
        pos_c = (cc["sentiment"] == "positive").sum()
        neg_c = (cc["sentiment"] == "negative").sum()
        neu_c = (cc["sentiment"] == "neutral").sum()
        total_sentiment = pos_c + neg_c + neu_c
        
        if total_sentiment > 0:
            pos_pct = (pos_c / total_sentiment) * 100
            neg_pct = (neg_c / total_sentiment) * 100
            neu_pct = (neu_c / total_sentiment) * 100
            sentiment_info = f"{pos_pct:.1f}% pos, {neg_pct:.1f}% neg, {neu_pct:.1f}% neu"
        else:
            sentiment_info = "no sentiment data"
        
        nodes.append({
            "id": node,
            "label": node,
            "title": f"{node}: {ccount} {title_prefix} ({sentiment_info})",
            "color": color,
            "size": size,
            "x": float(x * 5500),
            "y": float(y * 5500),
            "physics": False,
            "shape": "dot"
        })

    edges = []
    mx_w = agg_filtered["weight"].max() if len(agg_filtered) > 0 else 1
    for _, row in agg_filtered.iterrows():
        u, v = row["speaker"], row["target"]
        sentiment, w = row["sentiment"], row["weight"]
        width = 1 + 14 * (w / mx_w)
        
        # Calculate edge color using the new red-green-yellow formula
        pos_c = row["pos_count"]
        neg_c = row["neg_count"]
        neu_c = row["neu_count"]
        
        edge_color = remarks_to_color(neg_c, pos_c, neu_c)
        
        # Calculate percentages for tooltip
        total = pos_c + neg_c + neu_c
        if total > 0:
            pos_pct = (pos_c / total) * 100
            neg_pct = (neg_c / total) * 100
            neu_pct = (neu_c / total) * 100
            sentiment_info = f"{pos_pct:.1f}% pos, {neg_pct:.1f}% neg, {neu_pct:.1f}% neu"
        else:
            sentiment_info = "no sentiment data"
        
        edges.append({
            "from": u,
            "to": v,
            "color": edge_color,
            "width": width,
            "title": f"{w} remarks ({sentiment}) - {sentiment_info}",
            "arrows": {"to": {"enabled": True, "scaleFactor": 0.5, "type": "arrow"}}
        })

    return {"nodes": nodes, "edges": edges}

def build_graph_layout(max_book: int = 7):
    """Build NetworkX graph layout for positioning nodes."""
    df_filtered = df[df["book"] <= max_book]
    
    # Create filtered aggregation data
    agg_filtered = df_filtered.groupby(["speaker", "target"]).agg(
        weight=("sentiment", "size"),
        pos_count=("sentiment", lambda x: (x == "positive").sum()),
        neg_count=("sentiment", lambda x: (x == "negative").sum()),
        neu_count=("sentiment", lambda x: (x == "neutral").sum()),
    ).reset_index()
    
    if len(agg_filtered) == 0:
        return {}
    
    # Build graph (for layout)
    G = nx.from_pandas_edgelist(
        agg_filtered, source="speaker", target="target", edge_attr=["weight"], create_using=nx.DiGraph()
    )

    # Calculate positions using spring layout
    pos = nx.spring_layout(G, weight="weight", k=5.0, iterations=150, seed=42)
    
    return pos

print("✅ Graph data builder function defined")


✅ Graph data builder function defined


In [23]:
# ============ Generate Graph Data ============
print("📊 Generating data for both viewing modes and all books...")

# Generate layout positions based on all books for consistency
print("🗺️  Calculating graph layout for consistent node positioning...")
pos = build_graph_layout(7)  # Use all books for the base layout

# Generate data for all books (books 1-7)
speaker_data_by_book = {}
target_data_by_book = {}

for book in range(1, 8):
    print(f"📖 Generating data for books 1-{book}...")
    speaker_data_by_book[book] = build_graph_data("speaker", book)
    target_data_by_book[book] = build_graph_data("target", book)
    print(f"   Speaker mode: {len(speaker_data_by_book[book]['nodes'])} nodes, {len(speaker_data_by_book[book]['edges'])} edges")
    print(f"   Target mode: {len(target_data_by_book[book]['nodes'])} nodes, {len(target_data_by_book[book]['edges'])} edges")

# Keep the original data for backward compatibility (all books)
speaker_data = speaker_data_by_book[7]
target_data = target_data_by_book[7]

print(f"✅ Generated data for all book combinations")
print(f"📊 All books - Speaker mode: {len(speaker_data['nodes'])} nodes, {len(speaker_data['edges'])} edges")
print(f"📊 All books - Target mode: {len(target_data['nodes'])} nodes, {len(target_data['edges'])} edges")


📊 Generating data for both viewing modes and all books...
🗺️  Calculating graph layout for consistent node positioning...
📖 Generating data for books 1-1...
   Speaker mode: 440 nodes, 498 edges
   Target mode: 440 nodes, 498 edges
📖 Generating data for books 1-2...
   Speaker mode: 440 nodes, 845 edges
   Target mode: 440 nodes, 845 edges
📖 Generating data for books 1-3...
   Speaker mode: 440 nodes, 1224 edges
   Target mode: 440 nodes, 1224 edges
📖 Generating data for books 1-4...
   Speaker mode: 440 nodes, 1825 edges
   Target mode: 440 nodes, 1825 edges
📖 Generating data for books 1-5...
   Speaker mode: 440 nodes, 2537 edges
   Target mode: 440 nodes, 2537 edges
📖 Generating data for books 1-6...
   Speaker mode: 440 nodes, 2998 edges
   Target mode: 440 nodes, 2998 edges
📖 Generating data for books 1-7...
   Speaker mode: 440 nodes, 3630 edges
   Target mode: 440 nodes, 3630 edges
✅ Generated data for all book combinations
📊 All books - Speaker mode: 440 nodes, 3630 edges
📊 All

In [24]:
# ============ Generate HTML Visualization ============
print("🌐 Creating interactive HTML visualization...")

html_template = f"""
<html>
<head>
<meta charset="utf-8"/>
<script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
<style>
* {{
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}}
html, body {{
  height: 100%;
  overflow: hidden;
  font-family: sans-serif;
}}
.controls {{
  position: fixed;
  top: 10px;
  left: 10px;
  z-index: 1000;
  background: rgba(255,255,255,0.95);
  padding: 8px 12px;
  border-radius: 8px;
  box-shadow: 0 2px 8px rgba(0,0,0,0.15);
  display: flex;
  align-items: center;
  gap: 15px;
  max-width: 90vw;
}}
.filter-group {{
  display: flex;
  align-items: center;
  gap: 6px;
  position: relative;
}}
.filter-group label {{
  font-size: 12px;
  font-weight: bold;
  color: #555;
  white-space: nowrap;
}}
.search-container {{
  position: relative;
}}
.dropdown-container {{
  position: absolute;
  top: 100%;
  left: 0;
  right: 0;
  background: white;
  border: 1px solid #ccc;
  border-radius: 4px;
  box-shadow: 0 4px 12px rgba(0,0,0,0.15);
  max-height: 200px;
  overflow-y: auto;
  z-index: 1001;
  display: none;
}}
.dropdown-container.show {{
  display: block;
}}
.filter-controls {{
  display: flex;
  gap: 4px;
  align-items: center;
  margin-top: 8px;
  padding: 0 4px;
}}
button {{
  padding: 6px 12px;
  border-radius: 6px;
  border: 1px solid #ccc;
  background: #f7f7f7;
  cursor: pointer;
  font-size: 14px;
  white-space: nowrap;
}}
button:hover {{
  background: #e8f5e9;
}}
button.active {{
  background: #e8f5e9;
  border-color: #a5d6a7;
  font-weight: bold;
}}
select {{
  padding: 4px 8px;
  border-radius: 4px;
  border: 1px solid #ccc;
  font-size: 12px;
  width: 100%;
  background: white;
}}
select[multiple] {{
  min-height: 150px;
  border: none;
}}
.clear-btn {{
  padding: 4px 8px;
  font-size: 11px;
  background: #ffebee;
  border-color: #ffcdd2;
  min-width: auto;
}}
.clear-btn:hover {{
  background: #ffcdd2;
}}
.search-input {{
  padding: 6px 10px;
  border-radius: 4px;
  border: 1px solid #ccc;
  font-size: 12px;
  width: 180px;
  background: white;
}}
.search-input:focus {{
  outline: none;
  border-color: #a5d6a7;
  box-shadow: 0 0 0 2px rgba(165, 214, 167, 0.2);
}}
#network {{
  width: 100vw;
  height: 100vh;
  border: none;
}}
#legend-bubble {{
  position: fixed;
  right: 12px;
  bottom: 80px;
  background: rgba(255,255,255,0.95);
  border: 1px solid #ccc;
  border-radius: 10px;
  padding: 10px 14px;
  font: 13px/1.4 sans-serif;
  box-shadow: 0 2px 8px rgba(0,0,0,0.2);
  max-width: 320px;
  z-index: 1000;
}}
.timeline-container {{
  position: fixed;
  bottom: 0;
  left: 0;
  right: 0;
  height: 70px;
  background: rgba(255,255,255,0.95);
  border-top: 1px solid #ccc;
  box-shadow: 0 -2px 8px rgba(0,0,0,0.1);
  display: flex;
  align-items: center;
  padding: 0 20px;
  z-index: 1000;
  font-family: sans-serif;
}}
.timeline-content {{
  display: flex;
  align-items: center;
  width: 100%;
  max-width: 1200px;
  margin: 0 auto;
  gap: 20px;
}}
.timeline-label {{
  font-weight: bold;
  color: #333;
  white-space: nowrap;
  min-width: 120px;
}}
.timeline-slider-container {{
  flex: 1;
  display: flex;
  flex-direction: column;
  gap: 5px;
}}
.timeline-slider {{
  width: 100%;
  height: 8px;
  border-radius: 5px;
  background: #ddd;
  outline: none;
  cursor: pointer;
}}
.timeline-slider::-webkit-slider-thumb {{
  appearance: none;
  width: 20px;
  height: 20px;
  border-radius: 50%;
  background: #4CAF50;
  cursor: pointer;
  box-shadow: 0 2px 4px rgba(0,0,0,0.2);
}}
.timeline-slider::-moz-range-thumb {{
  width: 20px;
  height: 20px;
  border-radius: 50%;
  background: #4CAF50;
  cursor: pointer;
  box-shadow: 0 2px 4px rgba(0,0,0,0.2);
  border: none;
}}
.timeline-labels {{
  display: flex;
  justify-content: space-between;
  font-size: 11px;
  color: #666;
  margin-top: 2px;
}}
.book-info {{
  font-size: 14px;
  color: #333;
  font-weight: bold;
  min-width: 200px;
  text-align: right;
}}
</style>
</head>
<body>
<div class="controls">
  <button id="btnSpeaker" class="active" onclick="showGraph('speaker')">Speaker Graph</button>
  <button id="btnTarget" onclick="showGraph('target')">Target Graph</button>
  <button id="btnEdges" onclick="toggleEdges()">Hide Edges</button>
  
  <div class="filter-group">
    <label>Speakers:</label>
    <div class="search-container">
      <input type="text" class="search-input" id="speakerSearch" placeholder="Search speakers..." 
             oninput="filterSelectOptions('speakerSelect', this.value)"
             onfocus="showDropdown('speakerDropdown')" 
             onblur="hideDropdown('speakerDropdown')">
      <div id="speakerDropdown" class="dropdown-container">
        <select id="speakerSelect" multiple onchange="applyFilters()">
        </select>
        <div class="filter-controls">
          <button class="clear-btn" onmousedown="clearSpeakers()">Clear</button>
          <button class="clear-btn" onmousedown="selectAllSpeakers()">All</button>
        </div>
      </div>
    </div>
  </div>
  
  <div class="filter-group">
    <label>Targets:</label>
    <div class="search-container">
      <input type="text" class="search-input" id="targetSearch" placeholder="Search targets..." 
             oninput="filterSelectOptions('targetSelect', this.value)"
             onfocus="showDropdown('targetDropdown')"
             onblur="hideDropdown('targetDropdown')">
      <div id="targetDropdown" class="dropdown-container">
        <select id="targetSelect" multiple onchange="applyFilters()">
        </select>
        <div class="filter-controls">
          <button class="clear-btn" onmousedown="clearTargets()">Clear</button>
          <button class="clear-btn" onmousedown="selectAllTargets()">All</button>
        </div>
      </div>
    </div>
  </div>
</div>
<div id="network"></div>

<div id="legend-bubble"></div>

<div class="timeline-container">
  <div class="timeline-content">
    <div class="timeline-label">Book Timeline:</div>
    <div class="timeline-slider-container">
      <input type="range" id="bookSlider" class="timeline-slider" min="1" max="7" value="7" step="1" onchange="updateTimeline(this.value)" oninput="updateTimeline(this.value)">
      <div class="timeline-labels">
        <span>Book 1</span>
        <span>Book 2</span>
        <span>Book 3</span>
        <span>Book 4</span>
        <span>Book 5</span>
        <span>Book 6</span>
        <span>Book 7</span>
      </div>
    </div>
    <div class="book-info" id="bookInfo">All Books (1-7)</div>
  </div>
</div>

<script>
var speakerData = {json.dumps(speaker_data)};
var targetData = {json.dumps(target_data)};

// Data for each book (books 1-7)
var speakerDataByBook = {json.dumps(speaker_data_by_book)};
var targetDataByBook = {json.dumps(target_data_by_book)};

// Store current book selection
var currentBook = 7;

var allSpeakers = [...new Set(speakerData.edges.map(e => e.from))].sort();
var allTargets = [...new Set(speakerData.edges.map(e => e.to))].sort();

var legendSpeaker = `
  <strong>Graph Legend – Speaker Mode</strong><br>
  • Node size = # remarks <b>made</b><br>
  • Node color = sentiment mix of <b>remarks made</b><br>
  • Edge size = # remarks <b>made</b> about the targeted character<br>
  • Edge color = sentiment mix of <b>remarks made</b> about the targeted character<br><br>
  <strong>How to Use</strong><br>
  • Click node = show only its outgoing edges<br>
  • Double-click background = reset<br>
  • Filters: Select speakers only → show all their outgoing edges<br>
  • Filters: Select targets only → show all their incoming edges<br>
  • Filters: Select both → show only edges FROM speakers TO targets<br>
  • "Hide/Show Edges" = toggle all edges
`;

var legendTarget = `
  <strong>Graph Legend – Target Mode</strong><br>
  • Node size = # remarks <b>received</b><br>
  • Node color = sentiment mix of <b>received remarks</b><br>
  • Edge size = # remarks <b>received</b> by the character<br>
  • Edge color = sentiment mix of <b>received remarks</b> by the character<br><br>
  <strong>How to Use</strong><br>
  • Click node = show only its incoming edges<br>
  • Double-click background = reset<br>
  • Filters: Select speakers only → show all their outgoing edges<br>
  • Filters: Select targets only → show all their incoming edges<br>
  • Filters: Select both → show only edges FROM speakers TO targets<br>
  • "Hide/Show Edges" = toggle all edges
`;

var current = 'speaker';
var edgesHidden = false;
var container = document.getElementById('network');
var network, nodes, edges, originalNodesCopy, originalEdgesCopy;

function populateSelects() {{
  var speakerSelect = document.getElementById('speakerSelect');
  var targetSelect = document.getElementById('targetSelect');
  
  speakerSelect.innerHTML = '';
  targetSelect.innerHTML = '';
  
  allSpeakers.forEach(function(speaker) {{
    var option = document.createElement('option');
    option.value = speaker;
    option.textContent = speaker;
    speakerSelect.appendChild(option);
  }});
  
  allTargets.forEach(function(target) {{
    var option = document.createElement('option');
    option.value = target;
    option.textContent = target;
    targetSelect.appendChild(option);
  }});
}}

function showDropdown(dropdownId) {{
  document.getElementById(dropdownId).classList.add('show');
}}

function hideDropdown(dropdownId) {{
  // Use setTimeout to allow for button clicks before hiding
  setTimeout(function() {{
    document.getElementById(dropdownId).classList.remove('show');
  }}, 150);
}}

function filterSelectOptions(selectId, searchTerm) {{
  var select = document.getElementById(selectId);
  var options = select.querySelectorAll('option');
  var selectedValues = Array.from(select.selectedOptions).map(opt => opt.value);
  
  options.forEach(function(option) {{
    if (option.textContent.toLowerCase().includes(searchTerm.toLowerCase())) {{
      option.style.display = '';
    }} else {{
      option.style.display = 'none';
    }}
  }});
  
  // Restore selected values after filtering
  setTimeout(function() {{
    selectedValues.forEach(function(value) {{
      var opt = select.querySelector('option[value="' + value + '"]');
      if (opt) opt.selected = true;
    }});
  }}, 0);
}}

function drawGraph(data){{
  nodes = new vis.DataSet(data.nodes);
  edges = new vis.DataSet(data.edges);
  originalNodesCopy = new vis.DataSet(data.nodes);
  originalEdgesCopy = new vis.DataSet(data.edges);

  var options = {{
    interaction: {{ hover: true }},
    physics: false,
    nodes: {{ font: {{ size: 14 }} }}
  }};
  network = new vis.Network(container, {{nodes: nodes, edges: edges}}, options);

  // Click filtering
  network.on("selectNode", function(params){{
    if(params.nodes.length){{
      var nodeId = params.nodes[0];
      var keepEdges;
      if(current === 'speaker'){{
        keepEdges = originalEdgesCopy.get({{filter: function(e){{return e.from===nodeId;}}}});
      }} else {{
        keepEdges = originalEdgesCopy.get({{filter: function(e){{return e.to===nodeId;}}}});
      }}
      var nbrs = new Set([nodeId]);
      keepEdges.forEach(function(e){{nbrs.add(e.from); nbrs.add(e.to);}});
      var updates = nodes.get().map(function(n){{return {{id:n.id, hidden:!nbrs.has(n.id)}};}});
      edges.clear(); edges.add(keepEdges); nodes.update(updates);
    }}
  }});
  network.on("doubleClick", function(params){{ resetAll(); }});
}}

function applyFilters() {{
  // Check if graph is initialized
  if (!nodes || !edges || !originalNodesCopy || !originalEdgesCopy) {{
    return; // Graph not initialized yet, skip filtering
  }}
  
  var speakerSelect = document.getElementById('speakerSelect');
  var targetSelect = document.getElementById('targetSelect');
  
  var selectedSpeakers = Array.from(speakerSelect.selectedOptions).map(opt => opt.value);
  var selectedTargets = Array.from(targetSelect.selectedOptions).map(opt => opt.value);
  
  var filteredNodes = new Set();
  var filteredEdges = [];
  
  if (selectedSpeakers.length === 0 && selectedTargets.length === 0) {{
    // No filters applied - show all
    resetAll();
    return;
  }}
  
  // Add selected speakers to visible nodes
  selectedSpeakers.forEach(speaker => filteredNodes.add(speaker));
  
  // Add selected targets to visible nodes
  selectedTargets.forEach(target => filteredNodes.add(target));
  
  // Handle edges based on user requirements:
  // - Show only edges FROM selected speakers TO selected targets
  originalEdgesCopy.get().forEach(function(edge) {{
    var includeEdge = false;
    
    // If both speakers and targets are selected, show only edges between them
    if (selectedSpeakers.length > 0 && selectedTargets.length > 0) {{
      if (selectedSpeakers.includes(edge.from) && selectedTargets.includes(edge.to)) {{
        includeEdge = true;
      }}
    }}
    // If only speakers are selected, show all their outgoing edges
    else if (selectedSpeakers.length > 0 && selectedTargets.length === 0) {{
      if (selectedSpeakers.includes(edge.from)) {{
        includeEdge = true;
      }}
    }}
    // If only targets are selected, show all their incoming edges
    else if (selectedSpeakers.length === 0 && selectedTargets.length > 0) {{
      if (selectedTargets.includes(edge.to)) {{
        includeEdge = true;
      }}
    }}
    
    if (includeEdge) {{
      filteredNodes.add(edge.from);
      filteredNodes.add(edge.to);
      filteredEdges.push(edge);
    }}
  }});
  
  // Update the graph
  var nodeUpdates = originalNodesCopy.get().map(function(node) {{
    return {{
      id: node.id,
      hidden: !filteredNodes.has(node.id)
    }};
  }});
  
  nodes.update(nodeUpdates);
  edges.clear();
  edges.add(filteredEdges);
}}

function resetAll(){{
  // Check if graph is initialized
  if (!nodes || !edges || !originalNodesCopy || !originalEdgesCopy) {{
    return; // Graph not initialized yet, skip reset
  }}
  nodes.update(originalNodesCopy.get().map(function(n){{return {{id:n.id, hidden:false}};}}));
  edges.clear(); edges.add(originalEdgesCopy.get());
}}

function clearSpeakers() {{
  var select = document.getElementById('speakerSelect');
  for (var i = 0; i < select.options.length; i++) {{
    select.options[i].selected = false;
  }}
  document.getElementById('speakerSearch').value = '';
  filterSelectOptions('speakerSelect', '');
  applyFilters();
  // Close dropdown after clearing
  setTimeout(function() {{
    document.getElementById('speakerDropdown').classList.remove('show');
  }}, 100);
}}

function clearTargets() {{
  var select = document.getElementById('targetSelect');
  for (var i = 0; i < select.options.length; i++) {{
    select.options[i].selected = false;
  }}
  document.getElementById('targetSearch').value = '';
  filterSelectOptions('targetSelect', '');
  applyFilters();
  // Close dropdown after clearing
  setTimeout(function() {{
    document.getElementById('targetDropdown').classList.remove('show');
  }}, 100);
}}

function selectAllSpeakers() {{
  var select = document.getElementById('speakerSelect');
  for (var i = 0; i < select.options.length; i++) {{
    if (select.options[i].style.display !== 'none') {{
      select.options[i].selected = true;
    }}
  }}
  applyFilters();
  // Close dropdown after selecting all
  setTimeout(function() {{
    document.getElementById('speakerDropdown').classList.remove('show');
  }}, 100);
}}

function selectAllTargets() {{
  var select = document.getElementById('targetSelect');
  for (var i = 0; i < select.options.length; i++) {{
    if (select.options[i].style.display !== 'none') {{
      select.options[i].selected = true;
    }}
  }}
  applyFilters();
  // Close dropdown after selecting all
  setTimeout(function() {{
    document.getElementById('targetDropdown').classList.remove('show');
  }}, 100);
}}

function updateTimeline(bookValue) {{
  currentBook = parseInt(bookValue);
  var bookNames = [
    "", "Philosopher's Stone", "Chamber of Secrets", "Prisoner of Azkaban", 
    "Goblet of Fire", "Order of the Phoenix", "Half-Blood Prince", "Deathly Hallows"
  ];
  
  if (currentBook === 7) {{
    document.getElementById("bookInfo").textContent = "All Books (1-7)";
  }} else {{
    document.getElementById("bookInfo").textContent = `Book ${{currentBook}}: ${{bookNames[currentBook]}}`;
  }}
  
  // Get the appropriate data for the current book
  var currentSpeakerData = speakerDataByBook[currentBook] || speakerData;
  var currentTargetData = targetDataByBook[currentBook] || targetData;
  
  // Redraw the graph with filtered data
  if (current === 'speaker') {{
    drawGraph(currentSpeakerData);
  }} else {{
    drawGraph(currentTargetData);
  }}
  
  // Update speaker and target lists for filtering
  var newSpeakers = [...new Set(currentSpeakerData.edges.map(e => e.from))].sort();
  var newTargets = [...new Set(currentSpeakerData.edges.map(e => e.to))].sort();
  
  allSpeakers = newSpeakers;
  allTargets = newTargets;
  
  // Update the filter dropdowns
  populateSelects();
  clearSpeakers();
  clearTargets();
}}

function showGraph(which){{
  current = which;
  document.getElementById('btnSpeaker').classList.remove('active');
  document.getElementById('btnTarget').classList.remove('active');
  document.getElementById('btn'+which.charAt(0).toUpperCase()+which.slice(1)).classList.add('active');
  edgesHidden = false;
  document.getElementById("btnEdges").innerText = "Hide Edges";
  
  // Get the appropriate data for the current book
  var currentSpeakerData = speakerDataByBook[currentBook] || speakerData;
  var currentTargetData = targetDataByBook[currentBook] || targetData;
  
  if(which==='speaker') {{
    drawGraph(currentSpeakerData);
    document.getElementById("legend-bubble").innerHTML = legendSpeaker;
  }} else {{
    drawGraph(currentTargetData);
    document.getElementById("legend-bubble").innerHTML = legendTarget;
  }}
  
  // Clear filters and close dropdowns after drawing the graph
  clearSpeakers();
  clearTargets();
  // Ensure dropdowns are closed when switching modes
  document.getElementById('speakerDropdown').classList.remove('show');
  document.getElementById('targetDropdown').classList.remove('show');
}}

function toggleEdges(){{
  if(!edgesHidden){{
    edges.clear();
    document.getElementById("btnEdges").innerText = "Show Edges";
    edgesHidden = true;
  }} else {{
    edges.clear(); edges.add(originalEdgesCopy.get());
    document.getElementById("btnEdges").innerText = "Hide Edges";
    edgesHidden = false;
  }}
}}

// Initialize
populateSelects();
showGraph('speaker');
</script>
</body>
</html>
"""

# Write the HTML file
output_file = "index.html"
Path(output_file).write_text(html_template, encoding="utf-8")

print(f"✅ Interactive visualization saved as '{output_file}'")
print("🌐 Open the HTML file in your browser to explore the character relationships!")
print("📖 Use the buttons to switch between Speaker and Target modes")
print("🖱️  Click nodes to filter connections, double-click background to reset")


🌐 Creating interactive HTML visualization...
✅ Interactive visualization saved as 'index.html'
🌐 Open the HTML file in your browser to explore the character relationships!
📖 Use the buttons to switch between Speaker and Target modes
🖱️  Click nodes to filter connections, double-click background to reset
